In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
len(pt_features[pt_features['isCase']==False])

10975

In [9]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,non_insomnia_GP_consultations,stroke,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,antidepressants_pdds:0,antidepressants_pdds:1_10,antidepressants_pdds:11_100,antidepressants_pdds:101_1000,antidepressants_pdds:1001_10000,antidepressants_pdds:above_10000,sgas_pdds:0,sgas_pdds:1_10,sgas_pdds:11_100,sgas_pdds:101_1000,sgas_pdds:1001_10000,sgas_pdds:above_10000,fgas_pdds:0,fgas_pdds:1_10,fgas_pdds:11_100,fgas_pdds:101_1000,fgas_pdds:1001_10000,fgas_pdds:above_10000,other_sedatives_pdds:0,other_sedatives_pdds:1_10,other_sedatives_pdds:11_100,other_sedatives_pdds:101_1000,other_sedatives_pdds:1001_10000,other_sedatives_pdds:above_10000,benzo_and_z_drugs_pdds:0,benzo_and_z_drugs_pdds:1_10,benzo_and_z_drugs_pdds:11_100,benzo_and_z_drugs_pdds:101_1000,benzo_and_z_drugs_pdds:1001_10000,benzo_and_z_drugs_pdds:above_10000,mood_stabilisers_pdds:0,mood_stabilisers_pdds:1_10,mood_stabilisers_pdds:11_100,mood_stabilisers_pdds:101_1000,mood_stabilisers_pdds:1001_10000,mood_stabilisers_pdds:above_10000,benzo_and_z_drugs,mood_stabilisers_pdds,benzo_and_z_drugs_pdds,other_sedatives_pdds,fgas_pdds,sgas_pdds,sga_depots_pdds,fga_depots_pdds,antidepressants_pdds,insomnia,insomnia_any,insomnia_count:0,insomnia_count:1_10,insomnia_count:above_10,non_insomnia_GP_consultations:0
3801,19041089,17,89,1,2005-09-22,False,NaN,1997-07-25,2008-10-15,57669,88,55,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219.36913,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0
15676,4002447,41,447,0,2008-03-04,False,NaN,1999-12-15,2013-04-10,43226,67,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0
6674,18201274,35,274,0,2008-03-19,False,NaN,1999-10-25,2011-06-20,45527,73,23,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0
16278,343460,19,460,1,2008-01-21,False,NaN,1999-11-18,2013-04-18,28495,89,62,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0
8587,1153439,31,439,1,2008-02-07,False,NaN,1999-10-05,2011-09-21,35145,77,99,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0


In [10]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'CHD_heart_failure_and_peripheral_vascular_disease', 'hypertension',
       'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
       'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
       'age_at_index_date:65-69', 'age_at_index_date:70-74',
       'age_at_index_date:75-79', 'age_at_index_date:80-84',
       'age_at_index_date:85-89', 'age_at_index_date:90-99',
       'age_at_index_date:above_99', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'antidepressants_pdds:0',
       'antidepressants_pdds:1_10', 'antidepressants_pdds:11_100',
       'antidepressants_pdds:101_1000', 'antidepressants_pdds:1001_10000',
       'anti

In [11]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [12]:
baseline_characteristics = pd.DataFrame(index=characteristics,columns=['Cases','Controls','P value'])

In [13]:
baseline_dichot,baseline_contin = add_baseline_characteristics(baseline_characteristics,characteristics,pt_features)

In [14]:
baseline_contin

,Cases - mean,Cases - median,Cases - IQR,Controls - mean,Controls - median,Controls - IQR,P value,test
age_at_index_date,82.26,83.00,78.0 - 87.0,82.26,83.00,78.0 - 87.0,1.000,2-sample-t-test
non_insomnia_GP_consultations,75.22,66.00,38.0 - 100.0,67.65,60.00,33.0 - 91.0,0.000,2-sample-t-test
mood_stabilisers_pdds,22.13,0.00,0.0 - 0.0,10.95,0.00,0.0 - 0.0,0.000,2-sample-t-test
benzo_and_z_drugs_pdds,74.66,0.00,0.0 - 0.0,62.18,0.00,0.0 - 0.0,0.019,2-sample-t-test
other_sedatives_pdds,8.66,0.00,0.0 - 0.0,9.18,0.00,0.0 - 0.0,0.727,2-sample-t-test
fgas_pdds,18.82,0.00,0.0 - 0.0,13.04,0.00,0.0 - 0.0,0.048,2-sample-t-test
sgas_pdds,2.00,0.00,0.0 - 0.0,0.98,0.00,0.0 - 0.0,0.164,2-sample-t-test
fga_depots_pdds,0.04,0.00,0.0 - 0.0,0.23,0.00,0.0 - 0.0,0.186,2-sample-t-test
antidepressants_pdds,98.64,0.00,0.0 - 0.0,60.73,0.00,0.0 - 0.0,0.000,2-sample-t-test
insomnia,0.25,0.00,0.0 - 0.0,0.18,0.00,0.0 - 0.0,0.000,2-sample-t-test


In [15]:
baseline_dichot

,Cases,%Cases,Controls,%Controls,P value,test
female,7030.0,64.1%,7030.0,64.1%,1.000,chi2
stroke,1037.0,9.4%,775.0,7.1%,0.000,chi2
CHD_heart_failure_and_peripheral_vascular_disease,2755.0,25.1%,2500.0,22.8%,0.000,chi2
hypertension,4523.0,41.2%,4743.0,43.2%,0.003,chi2
diabetes,1073.0,9.8%,887.0,8.1%,0.000,chi2
mental_illness_non_smi,3528.0,32.1%,2472.0,22.5%,0.000,chi2
mental_illness_smi,596.0,5.4%,382.0,3.5%,0.000,chi2
sleep_apnoea,19.0,0.2%,17.0,0.2%,0.739,chi2
chronic_pulmonary_disease,2022.0,18.4%,1881.0,17.1%,0.013,chi2
epilepsy,188.0,1.7%,120.0,1.1%,0.000,chi2
